In [1]:
import argparse
import numpy as np
import torch
import torch.nn.functional as F
import pickle
import platform
import os

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
!git clone https://github.com/vita-epfl/DLAV-2023.git
path = os.getcwd() + '/DLAV-2023/homeworks/hw2/part1/test_batch'

Cloning into 'DLAV-2023'...


In [4]:
# Write the location of the saved weight relative to this notebook. Assume that they are in the same directory
### Path to Model Weights 
pytorch_weights = "./linearClassifier_pytorch.ckpt"

**TODO:** Copy the model you created from the Pytorch Notebook

In [5]:
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()

        # input layer
        self.fc1 = torch.nn.Linear(n_feature, n_hidden[0])
        # 2 hidden layers, number of nodes specified by n_hidden
        self.fc2 = torch.nn.Linear(n_hidden[0], n_hidden[1])
        self.fc3 = torch.nn.Linear(n_hidden[1], n_hidden[2])
        # Output layer, 10 units - one for each class
        self.fc4 = torch.nn.Linear(n_hidden[2], n_output)


    def forward(self, x):
        x = x.view(x.size(0),-1)
        # forward pass of input layer, activation func: ReLu
        x = self.fc1(x)
        x = F.relu(x)
        # forward pass of hidden layer, activation func: ReLu
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        # forward pass of output layer, activation func: ReLu
        x = F.relu(x)
        x = self.fc4(x)
        return x

**TODO**: Follow the instructions in each of the following methods. **Note that these methods should return a 1-D array of size N where N is the number of data samples. The values should be the predicted classes [0,...,9].**



In [10]:
def predict_usingPytorch(X):
    #########################################################################
    # TODO:                                                                 #
    # - Create your model                                                   #
    # - Load your saved model                                               #
    # - Do the operation required to get the predictions                    #
    # - Return predictions in a numpy array (hint: return "argmax")         #
    #########################################################################
    net = Net(n_feature=32*32*3, n_hidden=[128, 128, 64], n_output=10)     # create the model
    net.load_state_dict(torch.load(pytorch_weights)) # load the model
    outputs = net(X) # predict use the model
    _, y_pred = torch.max(F.softmax(outputs).data, 1) # get the predicted model
    #########################################################################
    #                       END OF YOUR CODE                                #
    #########################################################################
    return y_pred.numpy()


This method loads the test dataset to evaluate the model.

In [11]:
## Read DATA
def load_pickle(f):
    version = platform.python_version_tuple()
    if version[0] == '2':
        return  pickle.load(f)
    elif version[0] == '3':
        return  pickle.load(f, encoding='latin1')
    raise ValueError("invalid python version: {}".format(version))

def load_CIFAR_batch(filename):
  """ load single batch of cifar """
  with open(filename, 'rb') as f:
    datadict = load_pickle(f)
    X = datadict['data']
    Y = datadict['labels']
    X = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")
    Y = np.array(Y)
    return X, Y
test_filename = path
X,Y = load_CIFAR_batch(test_filename)

This code snippet prepares the data for the model. If you modify data manipulation in your notebooks, make sure to include them here. 

In [12]:
## Data Manipulation

mean = np.array([0.4914, 0.4822, 0.4465])
std = np.array([0.2023, 0.1994, 0.2010])
X = np.divide(np.subtract( X/255 , mean[np.newaxis,np.newaxis,:]), std[np.newaxis,np.newaxis,:])

X_pytorch = torch.Tensor(np.moveaxis(X,-1,1))


Runs evaluation on the Pytorch model. **Be careful that *prediction_pytorch* and *prediction_softmax* are 1-D array of size N where N is the number of data samples. The values should be the predicted class [0,...,9]**

---



In [15]:
## Run Prediction
prediction_pytorch = predict_usingPytorch(X_pytorch)

## Run Evaluation
acc_pytorch = sum(prediction_pytorch == Y)/len(X)
print("Softmax= %f ... Pytorch= %f"%(prediction_pytorch, acc_pytorch))

<ipython-input-10-c8ddc16018a7>:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  _, y_pred = torch.max(F.softmax(outputs).data, 1) # get the predicted model


TypeError: only size-1 arrays can be converted to Python scalars

In [14]:
acc_pytorch

0.5365